# 예측 데이터 셋

In [1]:
import pandas as pd
import numpy as np

# ----------------------------
# 역 정보
# ----------------------------
stations = [
    ["노포역", 35.2838438093648, 129.09502424558],
    ["사상역", 35.1625682832279, 128.988724714287],
    ["괴정역", 35.0999358214704, 128.992487217814],
    ["범내골역", 35.1473134465248, 129.05924810527],
    ["서면역", 35.157939062512, 129.059300883545],
    ["부산진화물역", 35.124819326380845, 129.04761201930125],
    ["부산역", 35.1145412381182, 129.039344263316],
    ["동대신역", 35.11032054771603, 129.0177333285286],
    ["서대신역", 35.11091202151769, 129.01213289704364],
    ["자갈치역", 35.097195508119285, 129.0260699211048],
    ["남포역", 35.09790554832192, 129.03473562824252],
    ["토성역", 35.1007852525409, 129.019785649708],
    ["양정역", 37.604027962835566, 127.19466908658632],
    ["부전역", 35.1646514052361, 129.060119800439],
    ["부산대역", 35.22964948660202, 129.08942537773808],
    ["온천장역", 35.2202705693472, 129.086435133803],
    ["하단역", 35.1062385683347, 128.966786546793],
    ["덕천역", 35.210434749918534, 129.00517512816737],
    ["구포역", 35.2054809460664, 128.9971139605],
    ["장산역", 35.1699558313015, 129.177022545045],
    ["센텀시티역", 35.168948888080884, 129.1317651180283],
    ["해운대역", 35.1636479638612, 129.158897240251],
    ["광안역", 35.157858583447, 129.113145418291],
    ["동래역", 35.20561606540984, 129.0784743842795],
    ["수영역", 35.165680361287, 129.114809550972],
    ["대연역", 35.1351251955716, 129.092118524312],
    ["경성대.부경대역", 35.1376003833537, 129.100519243797],
    ["연산역", 36.212295849098574, 127.20081634214654],
    ["남천역", 35.1420150673392, 129.107838322896],
    ["거제역", 35.18848999628461, 129.0740928144452],
    ["김해시청역", 35.2271218680623, 128.890387960522],
    ["불암역", 35.222144212820545, 128.92783103353437]
]

# ----------------------------
# 임의 승객 수 생성
# ----------------------------
# 붐비는 역
busy_stations = ["서면역", "부산역", "해운대역", "부산대역", "센텀시티역", "부전역"]
# 중간 역
medium_stations = ["동래역", "광안역", "수영역", "동대신역", "사상역", "양정역"]
# 나머지 작은 역
small_stations = [s[0] for s in stations if s[0] not in busy_stations + medium_stations]

data = []
np.random.seed(42)

for s in stations:
    name = s[0]
    lat = s[1]
    lng = s[2]
    
    if name in busy_stations:
        passengers = np.random.randint(20000, 50000)  # 붐비는 역
    elif name in medium_stations:
        passengers = np.random.randint(10000, 20000)  # 중간 역
    else:
        passengers = np.random.randint(5000, 10000)   # 작은 역
        
    data.append([name, lat, lng, passengers])

# ----------------------------
# 데이터프레임 생성
# ----------------------------
df = pd.DataFrame(data, columns=["역명", "위도", "경도", "승객수"])
df.to_csv("busan_subway_passengers.csv", index=False, encoding="utf-8-sig")
print(df.head())


     역명         위도          경도    승객수
0   노포역  35.283844  129.095024   5860
1   사상역  35.162568  128.988725  15390
2   괴정역  35.099936  128.992487   8772
3  범내골역  35.147313  129.059248   8092
4   서면역  35.157939  129.059301  42118


# 시각화

In [2]:
# ----------------------------
# 1. 라이브러리 불러오기
# ----------------------------
import pandas as pd  # CSV 읽기, 데이터 처리
import folium  # 지도 시각화 라이브러리

# ----------------------------
# 2. 데이터 불러오기
# ----------------------------
df = pd.read_csv("busan_subway_passengers.csv", encoding="utf-8")
# CSV 파일 읽기, 한글 깨짐 방지

# ----------------------------
# 3. 색상 기준 함수
# ----------------------------
def get_color(passengers):
    """승객 수에 따라 CircleMarker 색상 결정"""
    if passengers < 10000:
        return "blue"    # 적은 승객
    elif passengers < 30000:
        return "orange"  # 중간 승객
    else:
        return "red"     # 많은 승객

# ----------------------------
# 4. 반경 크기 조정 함수
# ----------------------------
def scale_radius(passengers):
    """승객 수에 비례하여 원 크기 결정, 최소 5 최대 30"""
    r = passengers / 2000
    return max(5, min(r, 30))

# ----------------------------
# 5. 지도 생성 (부산 중심 좌표)
# ----------------------------
m = folium.Map(location=[35.1796, 129.0756], zoom_start=12)
# 부산 중심 좌표, zoom_start=12 정도로 도시 전체 표시

# ----------------------------
# 6. 역별 승객수 지도 표시
# ----------------------------
threshold = 30000  # 붐비는 역 기준

for idx, row in df.iterrows():
    lat = row['위도']
    lng = row['경도']
    name = row['역명']

    # CSV의 실제 승객수 사용
    passengers = row["승객수"]

    # 색상과 반경 결정
    color = get_color(passengers)
    radius = scale_radius(passengers)

    # 팝업 HTML 예쁘게 꾸미기
    popup_html = f"""
    <div style="
        font-size:14px;
        font-weight:bold;
        color:#333;
        background-color:rgba(255,255,255,0.9);
        padding:6px 10px;
        border-radius:6px;
        box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
        text-align:center;
    ">
        <div>{name}</div>
        <div>승객수: {int(passengers):,}</div>
    </div>
    """

    # 모든 역 CircleMarker 표시
    folium.CircleMarker(
        location=[lat, lng],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=folium.Popup(popup_html, max_width=250)
    ).add_to(m)

    # 붐비는 역만 이름 표시
    if passengers >= threshold:
        folium.map.Marker(
            location=[lat, lng],
            icon=folium.DivIcon(
                html=f"""
                <div style="
                    display:inline-block;
                    white-space:nowrap;  /* 글자가 한 줄로 표시되도록 */
                    font-size:11px;      /* 글자 작게 조정 */
                    color:black;
                    font-weight:bold;
                    text-shadow: 1px 1px 2px white; /* 지도 위에서 글자 선명하게 */
                    background-color: rgba(255,255,255,0.6); /* 반투명 배경 */
                    padding: 2px 6px;
                    border-radius: 4px;
                ">
                {name}
                </div>
                """
            )
        ).add_to(m)

# ----------------------------
# 7. 지도 저장
# ----------------------------
m.save("busan_subway_map.html")
print("✅ 지도 시각화 완료! 'busan_subway_map.html' 파일을 열어보세요.")


✅ 지도 시각화 완료! 'busan_subway_map.html' 파일을 열어보세요.
